# Data science in Python

- Course GitHub repo: https://github.com/pycam/python-data-science
- Python website: https://www.python.org/ 

## Session 2.3: Biological data with BioPython


- [BioPython](#BioPython)
- [Working with sequences](#Working-with-sequences)
- [Connecting with biological databases](#Connecting-with-biological-databases)
- [Exercise 2.3.1](#Exercise-2.3.1)

## BioPython

The goal of Biopython is to make it as easy as possible to use Python for bioinformatics by creating high-quality, reusable modules and classes. Biopython features include parsers for various Bioinformatics file formats (BLAST, Clustalw, FASTA, Genbank,...), access to online services (NCBI, Expasy,...), interfaces to common and not-so-common programs (Clustalw, DSSP, MSMS...), a standard sequence class, various clustering modules, a KD tree data structure etc. and documentation as well as a tutorial: http://biopython.org/DIST/docs/tutorial/Tutorial.html.

The Biopython Application Programming Interface (API) documentation: http://biopython.org/DIST/docs/api/.

## Working with sequences

We can create a sequence by defining a `Seq` object with strings. `Bio.Seq()` takes as input a string and converts in into a Seq object. We can print the sequences, individual residues, lengths and use other functions to get summary statistics.  

In [3]:
# Creating sequence
from Bio.Seq import Seq
my_seq = Seq("AGTACACTGGT")
print(my_seq)
print(my_seq[10])
print(my_seq[1:5])
print(len(my_seq))
print(my_seq.count("A"))

AGTACACTGGT
T
GTAC
11
3


We can use functions from `Bio.SeqUtils` to get idea about a sequence 

In [4]:
# Calculate the molecular weight
from Bio.SeqUtils import GC, molecular_weight
print(GC(my_seq))
print(molecular_weight(my_seq))

45.45454545454545
3436.1957000000007


One letter code protein sequences can be converted into three letter codes using `seq3` utility 

In [5]:
from Bio.SeqUtils import seq3
print(seq3(my_seq))

AlaGlyThrAlaCysAlaCysThrGlyGlyThr


Alphabets defines how the strings are going to be treated as sequence object. `Bio.Alphabet` module defines the available alphabets for Biopython. `Bio.Alphabet.IUPAC` provides basic definition for DNA, RNA and proteins. 

In [6]:
from Bio.Alphabet import IUPAC
my_dna = Seq("AGTACATGACTGGTTTAG", IUPAC.unambiguous_dna)
print(my_dna)
print(my_dna.alphabet)

AGTACATGACTGGTTTAG
IUPACUnambiguousDNA()


In [7]:
my_dna.complement()

Seq('TCATGTACTGACCAAATC', IUPACUnambiguousDNA())

In [8]:
my_dna.reverse_complement()

Seq('CTAAACCAGTCATGTACT', IUPACUnambiguousDNA())

In [9]:
my_dna.translate()

Seq('ST*LV*', HasStopCodon(IUPACProtein(), '*'))

### Parsing sequence file format: FASTA files

Sequence files can be parsed and read the same way we read other files. 

In [ ]:
with open( "data/glpa.fa" ) as f:
    print(f.read())

In [ ]:
Biopython provides specific functions to allow parsing/reading sequence files. 

In [12]:
# Reading FASTA files
from Bio import SeqIO

with open("data/glpa.fa") as f:
    for protein in SeqIO.parse(f, 'fasta'):
        print(protein.id)
        print(protein.seq)

swissprot|P02724|GLPA_HUMAN
MYGKIIFVLLLSEIVSISASSTTGVAMHTSTSSSVTKSYISSQTNDTHKRDTYAATPRAHEVSEISVRTVYPPEEETGERVQLAHHFSEPEITLIIFGVMAGVIGTILLISYGIRRLIKKSPSDVKPLPSPDTDVPLSSVEIENPETSDQ


Sequence objects can be written into files using file handles with the function `SeqIO.write()`. We need to provide the name of the output sequence file and the sequence file format. 

In [ ]:
# Writing FASTA files
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC

sequence = 'MYGKIIFVLLLSEIVSISASSTTGVAMHTSTSSSVTKSYISSQTNDTHKRDTYAATPRAHEVSEISVRTVYPPEEETGERVQLAHHFSEPEITLIIFG'
 
seq = Seq(sequence, IUPAC.protein)
protein = [SeqRecord(seq, id="THEID", description='a description'),]
# **** force python to create a list by appending a comma ,  ****

with open( "biopython.fa", "w") as f:
    SeqIO.write(protein, f, 'fasta')

with open( "biopython.fa" ) as f:
    print(f.read())

## Connecting with biological databases

Sequences can be searched and downloaded from public databases. 

In [ ]:
# Read FASTA file from NCBI GenBank
from Bio import Entrez

Entrez.email = 'A.N.Other@example.com' # Always tell NCBI who you are
handle = Entrez.efetch(db="nucleotide", id="71066805", rettype="gb")
seq_record = SeqIO.read(handle, "gb")
handle.close()

print(seq_record.id, 'with', len(seq_record.features), 'features')
print(seq_record.seq)
print(seq_record.format("fasta"))

In [ ]:
# Read SWISSPROT record
from Bio import ExPASy

handle = ExPASy.get_sprot_raw('HBB_HUMAN')
prot_record = SeqIO.read(handle, "swiss")
handle.close()

print(prot_record.description)
print(prot_record.seq)

## Exercise 2.3.1

- Retrieve a FASTA file named `data/sample.fa` using BioPython and answer the following questions:
  - How many sequences are in the file?
  - What are the IDs and the lengths of the longest and the shortest sequences?
  - Select sequences longer than 500bp. What is the average length of these sequences?
  - Calculate and print the percentage of GC in each of the sequences.
  - Write the newly created sequences into a FASTA file named `long_sequences.fa` 

In [ ]:
from Bio import SeqIO

# read FASTA
seq_records = list(SeqIO.parse("data/sample.fa", 'fasta'))

# find the number of seqs present in the file
num_seq = len(seq_records)
print("total number of seqs", num_seq)

In [59]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqUtils import GC
import pandas as pd
protein_id=[]
protein_seq=[]
protein_length=[]
protein_GC=[]
with open("data/sample.fa") as f:
    for protein in SeqIO.parse(f, 'fasta'):
        protein_id.append(protein.id)
        protein_seq.append(protein.seq)
        protein_length.append(len(protein.seq))
        protein_GC.append(GC(protein.seq))
        #print(protein_id)
        #print(len(protein.seq))

df= pd.DataFrame({"id" : protein_id, "seq" : protein_seq, "length": protein_length})
df.head()
# # longest seq
# print('longest seq is',df.sort_values("length",ascending=False)["id"].iloc[0])
# # shortest seq
# print('longest seq is',df.sort_values("length")["id"].iloc[0])

# better way
longest = df.sort_values("length",ascending=False).iloc[0]
print('longest seq is', longest["id"], ' with length', longest["length"], 'bp')

# seq longer than 500bp
long_df = df[df["length"] > 500 ]
#print(long_df.describe())
print(long_df["length"].mean())

longest seq is seq82  with length 1006 bp
996.826086957


## Retrieving more biological data (live coding session)

## Next session

Go to our next notebook: [Session 2.4: Data project report in Jupyter](24_python_data.ipynb)